In [ ]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 1 para pegar movimentações de processos na busca do 1º Grau 
# a partir do campo "Nome da parte", um foro de cada vez
# Gera um arquivo com os links diretos de cada processo, que serão lidos no programa 2
#

## Chama bibliotecas e módulos

1 - BeautifulSoup transforma um documento HTML complexo em uma complexa árvore de objetos Python. Mas você só terá que lidar com quatro tipos de objetos - Tag, NavigableString, BeautifulSoup, and Comment

In [1]:
from bs4 import BeautifulSoup, Tag, NavigableString

2 - O BeautifulSoup 4.7+ usa uma biblioteca selecionada chamada Soup Sieve que é instalada ao lado do BeautifulSoup. Você pode importá-lo diretamente e criar padrões de correspondência de seletores CSS

In [2]:
import soupsieve as sv

3 - A biblioteca de requests faz solicitações HTTP em Python

In [3]:
import requests

4 - Expressões regulares

In [4]:
import re

5 - Pandas para datascience

In [5]:
import pandas as pd

6 - Retira acentuações

In [6]:
import unidecode

7 - Para converter strings em datas

In [7]:
from datetime import date

8 - Time para paralisar brevemente

In [8]:
import time

9 - Desabilita avisos

In [9]:
import urllib3; urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Funções

Função para retirar acentos

In [10]:
def f(str):
    return (unidecode.unidecode(str))

O tipo de processo não é agrupado em um elemento fácil de segmentar, mas segue o número do processo, portanto, podemos criar uma função simples para extrair o texto logo após a execução, até atingirmos a próxima div:

In [11]:
def get_process_type(el):
    # O texto é encontrado logo após esse filho, então colete o texto até chegarmos ao próximo div
    text = []
    # Você pode usar .next_sibling e .previous_sibling para navegar entre os elementos da página que estão no mesmo nível da árvore de análise
    sibling = el.next_sibling
    while True:
        sibling = sibling.next_sibling
        # A função isinstance() verifica se o objeto (primeiro argumento) é uma instância ou subclasse da classe classinfo(segundo argumento)
        # Uma string corresponde a um pouco de texto dentro de uma tag. BeautifulSoup usa a classe NavigableString para conter esses bits de texto
        if isinstance(sibling, NavigableString):
            text.append(sibling)
        # Um objeto Tag corresponde a uma tag XML ou HTML no documento original
        # Aqui não interessa
        elif isinstance(sibling, Tag):
            if sibling.name == 'div':
                break
    return ''.join(text).strip()

Função principal para ler sites e tags

In [12]:
def le_processos(link_tjsp1, link_tjsp2, link_tjsp3, link_tjsp4, nome, tribunal):
    print(" ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++")
    print(nome)
    
    # Com o uso de uma iteração percorre cada foro para fazer a busca
    for num, row in foro_sp.iterrows():
        foro_cd = str(row['codigo']) # código do foro
        foro = str((row['nome_foro']).strip()) # nome do foro
        print("Foro buscado: " + foro)
        
        # Une as partes dos links para cria o link de consulta da página
        link_consulta = link_tjsp1 + "1" + link_tjsp2 + foro_cd + link_tjsp3 + nome + link_tjsp4
        
        
        # Testa o link com o requests
        try:
            res = requests.get(link_consulta, verify=False) # evita o SSLError
        except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(str(e))
            return 
        except Exception as e:
            print("Exceção")
            return
        
        # Cria a sopa com a criação de uma parse tree - árvore de análise
        soup =  BeautifulSoup(res.text, "lxml")
    
        # Primeiro testa se a busca tem algum conteúdo    
        if soup.find('td', {'id': 'mensagemRetorno'}) is None:
          
            # seleciona a href da página final
            if soup.find_all('a', style ="padding: 5px;", title="Última página") == []:
                # Significa que a busca tem apenas 1 página
                pagina_final = 1
            else:
                # Busca a posição do número da página
                pagina_final = soup.find_all('a', {'style': 'padding: 5px;'})[1].get('href')
        
                # Seleciona a posicao inicial de interesse
                posicao1 = pagina_final.find('?paginaConsulta=')
                posicao2 = pagina_final[posicao1:posicao1+19]
        
                # retorna apenas o número
                pagina_final = int(''.join(list(filter(str.isdigit, posicao2))))
        
    
            # Inicia busca nas páginas, de acordo com o número de páginas
            for acao in range(1, pagina_final+1):
        
                #  Une as partes dos links para cria o link de consulta da página
                link_consulta = link_tjsp1 + str(acao) + link_tjsp2 + foro_cd + link_tjsp3 + nome + link_tjsp4
                
                print(link_consulta)
        
                # Testa o link com o requests
                try:
                    res = requests.get(link_consulta, verify=False) # evita SSLError
                except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
                    print(str(e))
                    return 
                except Exception as e:
                    print("Exceção")
                    return
        
                # Cria a sopa com a criação de uma parse tree - árvore de análise
                soup =  BeautifulSoup(res.text, "lxml")
        
                # Cria padrões CSS para encontrar dados que queremos capturar
                # Se você já usou a biblioteca re do Python para expressões regulares, talvez saiba que geralmente é útil pré-compilar um padrão de expressão regular, especialmente se você planeja usá-lo mais de uma vez

                # O mesmo vale para os matchers da Soup Sieve, embora não seja obrigatório

                # Se você tem um padrão que você deseja usar mais de uma vez, pode ser sábio pré-compilá-lo 
                legal_process = sv.compile('div.nuProcesso')
                info1 = sv.compile('br + div.espacamentoLinhas')
                info2 = sv.compile('div.espacamentoLinhas + div.espacamentoLinhas')
                
                # Expressão regular para separar informações de data e local
                date_split = re.compile(r'-\s+(?:\d+\xaa)?')
                # Checar: https://regex101.com
                # - matches the character - literally (case sensitive)
                # \s+ matches any whitespace character (equal to [\r\n\t\f\v ])
                # + Quantifier — Matches between one and unlimited times, as many times as possible, giving back as needed (greedy)

                # Non-capturing group (?:\d+\xaa)?
                # ? Quantifier — Matches between zero and one times, as many times as possible, giving back as needed (greedy)

                # \d+ matches a digit (equal to [0-9])
                # + Quantifier — Matches between one and unlimited times, as many times as possible, giving back as needed (greedy)
                # \xaa matches the character ª with index AA16 (17010 or 2528) literally (case sensitive)

                # Global pattern flags
                # g modifier: global. All matches (don't return after first match)
                # m modifier: multi line. Causes ^ and $ to match the begin/end of each line (not only begin/end of string)
                
                
                # Percorre os filhos contidos na tag
                for child in soup.select('div#listagemDeProcessos *'):
                    
                    # Se é a região com o número do processo
                    if legal_process.match(child):
                        # Guarda o número do processo na lista - dentro de um dicionário
                        processos.append({"process": child.text.strip()})
                
                        # Adiciona outro item, o link processo
                        # O -1 mantém o dicionário na mesma linha
                        processos[-1]['link'] = "https://esaj.tjsp.jus.br" + child.a.get('href')

                        # Guarda o tipo do processo
                        # Chama função 
                        processos[-1]['type'] = get_process_type(child)

                    # Se é a área com o tipo da parte no processo
                    elif info1.match(child):
                        # Armazene o rótulo e o nome necessários
                        processos[-1]['label'], processos[-1]['name'] = [item.strip() for item in child.text.split(':', 1)]
                    elif info2.match(child):
                        # Armazenar data e local
                        processos[-1]['date'], processos[-1]['location'] = [item.strip() for item in date_split.split(child.text.split(':', 1)[1], 1)]
    
        else: 
            # Não encontrou nada na busca
            print('-------------------------------------------------------------')
            print('A busca não foi encontrada para ' + nome + ' e foro ' + foro + ". Cheque manualmente")
            print('-------------------------------------------------------------')

## Execução do código

Acessa nome das partes

In [13]:
partes = pd.read_excel('dados/partes_buscas.xlsx', Sheet="Planilha1")

In [14]:
partes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
empresa_parte    3 non-null object
tribunal         3 non-null object
dtypes: object(2)
memory usage: 128.0+ bytes


In [15]:
partes.reset_index().head()

,index,empresa_parte,tribunal
0,0,Folha da Manhã,TJ-SP
1,1,Google,TJ-SP
2,2,Facebook,TJ-SP


Retira os acentos

In [16]:
partes["empresa_parte"] = partes["empresa_parte"].apply(f)

Acessa nomes dos foros de SP

In [17]:
foro_sp = pd.read_excel('dados/foros_justica_sao_paulo.xlsx', Sheet="Planilha2")

In [18]:
foro_sp

,codigo,nome_foro
0,91,ANTIGO Foro Distrital de Brás Cubas
1,509,Araçatuba/DEECRIM UR2
2,26,Bauru/DEECRIM UR3
3,502,Campinas/DEECRIM UR4
4,500,DEPRE
5,234,Foro - Corregedoria Geral da Justiça
6,53,Foro Central - Fazenda Pública/Acidentes
7,100,Foro Central Cível
8,52,Foro Central Criminal - Juri
9,50,Foro Central Criminal Barra Funda


Link principal para consulta de processos de primeiro grau

In [19]:
link_tjsp1 = 'https://esaj.tjsp.jus.br/cpopg/trocarPagina.do?paginaConsulta='
# vai o número da página depois
link_tjsp2 = '&conversationId=&dadosConsulta.localPesquisa.cdLocal='
# vai o código do foro depois
link_tjsp3 = '&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta='
# vai o nome da parte depois
link_tjsp4 = '&uuidCaptcha=&pbEnviar=Pesquisar'

Cria lista vazia para armazenar dados  

In [20]:
processos = []

Iteração para chamar a busca de cada parte

In [21]:
for num, row in partes.iterrows():
    nome = row['empresa_parte']
    tribunal = row['tribunal']
    time.sleep(5)
    le_processos(link_tjsp1, link_tjsp2, link_tjsp3, link_tjsp4, nome, tribunal)

 
++++++++++++++++++++++++++++++++++++++++++++++++
Folha da Manha
Foro buscado: ANTIGO Foro Distrital de Brás Cubas
https://esaj.tjsp.jus.br/cpopg/trocarPagina.do?paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=91&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha da Manha&uuidCaptcha=&pbEnviar=Pesquisar
Foro buscado: Araçatuba/DEECRIM UR2
https://esaj.tjsp.jus.br/cpopg/trocarPagina.do?paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=509&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha da Manha&uuidCaptcha=&pbEnviar=Pesquisar
Foro buscado: Bauru/DEECRIM UR3
https://esaj.tjsp.jus.br/cpopg/trocarPagina.do?paginaConsulta=1&conversationId=&dadosConsulta.localPesquisa.cdLocal=26&cbPesquisa=NMPARTE&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsulta=Folha da Manha&uuidCaptcha=&pbEnviar=Pesquisar
Foro buscado: Campinas/DEECRIM UR4
https://esaj.tjsp.jus.br

Converte o dicionário dentro da lista em um dataframe

# MAS AQUI PERCEBI QUE A PARTIR DE JUNHO DE 2019 A RASPAGEM NÃO ERA MAIS POSSÍVEL POR CAUSA DO RECAPTCHA - O DATAFRAME GERADO FICA VAZIO

In [22]:
df_processos = pd.DataFrame(processos, columns = ['process', 'link', 'type', 'label', 'name','date', 'location'])

In [23]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
process     0 non-null object
link        0 non-null object
type        0 non-null object
label       0 non-null object
name        0 non-null object
date        0 non-null object
location    0 non-null object
dtypes: object(7)
memory usage: 0.0+ bytes


Filtra pela data

In [25]:
df_processos['date'] = pd.to_datetime(df_processos['date'])

In [26]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 7 columns):
process     2458 non-null object
link        2458 non-null object
type        2458 non-null object
label       2458 non-null object
name        2458 non-null object
date        2333 non-null datetime64[ns]
location    2333 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 134.5+ KB


In [31]:
mask = (df_processos['date'] >= '2017-06-01') | (df_processos['date'] == '') # junho de 2017
df_processos_filtro = df_processos[mask]

Filtra pelo tipo da parte

In [32]:
df_processos_filtro = df_processos_filtro[(df_processos_filtro['label'] == 'Reqdo') | 
                                          (df_processos_filtro['label'] == 'Autor') | 
                                          (df_processos_filtro['label'] == 'Credor') | 
                                          (df_processos_filtro['label'] == 'Denunciado') | 
                                          (df_processos_filtro['label'] == 'Embargdo') | 
                                          (df_processos_filtro['label'] == 'Exectdo') | 
                                          (df_processos_filtro['label'] == 'Exeqte') | 
                                          (df_processos_filtro['label'] == 'Interesdo.') | 
                                          (df_processos_filtro['label'] == 'Perito') | 
                                          (df_processos_filtro['label'] == 'Recebido em') | 
                                          (df_processos_filtro['label'] == 'Reqda') | 
                                          (df_processos_filtro['label'] == 'Reqdo') | 
                                          (df_processos_filtro['label'] == 'Reqte') | 
                                          (df_processos_filtro['label'] == 'Réu') | 
                                          (df_processos_filtro['label'] == 'TerIntCer') | 
                                          (df_processos_filtro['label'] == 'TerIntInc') ]

In [34]:
df_processos_filtro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097 entries, 1 to 2457
Data columns (total 7 columns):
process     1097 non-null object
link        1097 non-null object
type        1097 non-null object
label       1097 non-null object
name        1097 non-null object
date        1097 non-null datetime64[ns]
location    1097 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 68.6+ KB


In [35]:
df_processos_filtro.reset_index().head()

,index,process,link,type,label,name,date,location
0,1,1005696-11.2018.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Anulação de Débito Fiscal,Reqte,Empresa Folha da Manhã S.A.,2018-06-02,Vara de Fazenda Pública
1,72,1029643-16.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Intimação / Notificação,Reqda,Empresa Folha da Manhã S.A.,2019-03-04,Vara Cível
2,73,1028972-90.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqte,Empresa Folha da Manhã S.A.,2019-01-04,Vara Cível
3,75,1020778-04.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Classificação de créditos,Reqte,Empresa Folha da Manhã S.A.,2019-11-03,Vara de Falências e Recuperações Judiciais
4,76,1004643-14.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqda,Empresa Folha da Manhã S.A.,2019-01-23,Vara Cível


Salva um arquivo estruturado

In [36]:
df_processos_filtro.to_excel('resultados/processos_teste_tjsp_22_abril.xlsx',sheet_name='Sheet1')